In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 

In [2]:
offers = pd.read_csv('datasets/data_offers.csv')
orders = pd.read_csv('datasets/data_orders.csv')

In [ ]:
df 